In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

import requests
from bs4 import BeautifulSoup
import pandas as pd
import itertools
import re
from tqdm import tqdm_notebook
import string
import numpy as np

# Удаление стоп слов и стемминг

In [45]:
df_final = pd.read_csv("coursera_clean.csv")
df_stack = pd.read_csv("tidy_questions.csv", index_col = "index", encoding = "utf-8", dtype={'index':np.str})

In [47]:
df_stack.drop("Unnamed: 0", axis = 1, inplace = True)

In [48]:
df_stack.dropna(inplace=True)

In [65]:
df_stack.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 288227 entries, 23755195 to 31632133
Data columns (total 1 columns):
Text    288227 non-null object
dtypes: object(1)
memory usage: 4.4+ MB


In [4]:
df_final.head()

,index,about,level,rating,rating_count,syllabus
0,https://www.coursera.org/learn/python,this course aims to teach everyone the basics...,NaN,4.8,29620,chapter one why we program these are the cours...
1,https://www.coursera.org/learn/algorithms-part1,this course covers the essential information ...,Intermediate,4.9,2648,course introduction welcome to algorithms part...
2,https://www.coursera.org/learn/python-data-ana...,this course will introduce the learner to the...,Intermediate,4.5,3836,week in this week youl get an introduction to ...
3,https://www.coursera.org/learn/algorithmic-too...,the course covers basic algorithmic technique...,NaN,4.6,2355,welcome welcome to the first module of data st...
4,https://www.coursera.org/learn/python-data,this course will introduce the core data stru...,NaN,4.8,16116,chapter six strings in this class we pick up w...


In [6]:
stop_words = set(stopwords.words('english'))

In [7]:
def stemm_stop_words(string):
    string = str(string)
    tokens = word_tokenize(string)
    
    ps = PorterStemmer()

    filtered_sentence = [w for w in tokens if not w in stop_words]
    
    stem_sentence = [ps.stem(w) for w in filtered_sentence]
    
    string = " ".join(stem_sentence)
    return string

In [8]:
df_final["about"] = df_final["about"].apply(stemm_stop_words)
df_final["syllabus"] = df_final["syllabus"].apply(stemm_stop_words)

In [9]:
df_final.head()

,index,about,level,rating,rating_count,syllabus
0,https://www.coursera.org/learn/python,cours aim teach everyon basic program comput u...,NaN,4.8,29620,chapter one program coursewid materi well firs...
1,https://www.coursera.org/learn/algorithms-part1,cours cover essenti inform everi seriou progra...,Intermediate,4.9,2648,cours introduct welcom algorithm part video ex...
2,https://www.coursera.org/learn/python-data-ana...,cours introduc learner basic python program en...,Intermediate,4.5,3836,week week youl get introduct field data scienc...
3,https://www.coursera.org/learn/algorithmic-too...,cours cover basic algorithm techniqu idea comp...,NaN,4.6,2355,welcom welcom first modul data structur algori...
4,https://www.coursera.org/learn/python-data,cours introduc core data structur python progr...,NaN,4.8,16116,chapter six string class pick left previou cla...


## Запись в цсв

In [10]:
df_final.to_csv("coursera.csv")

# kNN модель

In [6]:
df_final = pd.read_csv('coursera.csv').drop('Unnamed: 0',axis=1)
df_stack = pd.read_csv('tidy_questions.csv').drop('Unnamed: 0', axis=1).set_index('Id').dropna()

In [7]:
df_stack.head()

,Text
Id,
23755195,data export xml file reason use encod special ...
9441598,specif want gener cryptograph secur random num...
27978003,develop asp net mvc websit use jqueri ajax req...
1988012,write client applic queri url confirm user aut...
12370490,wcf servic host sharepoint retunr list object ...


In [8]:
df_final['level'] = df_final['level'].fillna('unknown')

In [9]:
df_final.head()

,index,about,level,rating,rating_count,syllabus
0,https://www.coursera.org/learn/python,cours aim teach everyon basic program comput u...,unknown,4.8,29620,chapter one program coursewid materi well firs...
1,https://www.coursera.org/learn/algorithms-part1,cours cover essenti inform everi seriou progra...,Intermediate,4.9,2648,cours introduct welcom algorithm part video ex...
2,https://www.coursera.org/learn/python-data-ana...,cours introduc learner basic python program en...,Intermediate,4.5,3836,week week youl get introduct field data scienc...
3,https://www.coursera.org/learn/algorithmic-too...,cours cover basic algorithm techniqu idea comp...,unknown,4.6,2355,welcom welcom first modul data structur algori...
4,https://www.coursera.org/learn/python-data,cours introduc core data structur python progr...,unknown,4.8,16116,chapter six string class pick left previou cla...


In [10]:
df_final['Text'] = df_final["about"] + df_final["syllabus"]
df_final['index'] = df_final['index'] + ' ' + df_final['level'] + ' ' + df_final['rating'].astype('str') + ' ' + df_final['rating_count'].astype('str')
df_train = df_final[["index", "Text"]]

In [11]:
df_train.head()

,index,Text
0,https://www.coursera.org/learn/python unknown ...,cours aim teach everyon basic program comput u...
1,https://www.coursera.org/learn/algorithms-part...,cours cover essenti inform everi seriou progra...
2,https://www.coursera.org/learn/python-data-ana...,cours introduc learner basic python program en...
3,https://www.coursera.org/learn/algorithmic-too...,cours cover basic algorithm techniqu idea comp...
4,https://www.coursera.org/learn/python-data unk...,cours introduc core data structur python progr...


In [12]:
df_train.set_index('index', drop = True, inplace=True)

In [13]:
df_train.head()

,Text
index,
https://www.coursera.org/learn/python unknown 4.8 29620,cours aim teach everyon basic program comput u...
https://www.coursera.org/learn/algorithms-part1 Intermediate 4.9 2648,cours cover essenti inform everi seriou progra...
https://www.coursera.org/learn/python-data-analysis Intermediate 4.5 3836,cours introduc learner basic python program en...
https://www.coursera.org/learn/algorithmic-toolbox unknown 4.6 2355,cours cover basic algorithm techniqu idea comp...
https://www.coursera.org/learn/python-data unknown 4.8 16116,cours introduc core data structur python progr...


In [14]:
df_train.to_csv('coursera_union.csv')

In [15]:
df_cnn = pd.concat([df_train, df_stack], axis=0)

In [16]:
df_cnn.to_csv('coursera_ques.csv')

In [17]:
df_cnn.tail(10)

,Text
24346956,pleas take look let know chang background colo...
33208055,could two php version differ path server curre...
14590341,easi get work run littl demo use googl homepag...
26497549,assign suppos creat rock paper scissor game us...
20965290,new facebook app develop found fb app develop ...
42507623,click button mobil chrome blue color appear se...
7167806,applic sign jar certif mine wan run applic web...
32203664,alreadi creat splash screen use jqueri fadeout...
9704650,know question seem imposs sure somebodi help d...
31632133,want build x data frame r use two vector valu ...


In [18]:
#Define the TFIDF vectorizer that will be used to process the data
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_cnn['Text'])
tfidf_courses = tfidf_vectorizer.transform(df_train['Text'])
tfidf_questions = tfidf_vectorizer.transform(df_stack['Text'])

In [19]:
tfidf_matrix.shape, tfidf_courses.shape, tfidf_questions.shape

((288760, 176039), (533, 176039), (288227, 176039))

In [20]:
nbrs = NearestNeighbors(n_neighbors=10).fit(tfidf_matrix)
nbrs_courses = NearestNeighbors(n_neighbors=10).fit(tfidf_courses)
nbrs_questions = NearestNeighbors(n_neighbors=10).fit(tfidf_questions)

In [21]:
nbrs

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=10, p=2, radius=1.0)

In [22]:
df_cnn.to_csv('coursera_ques.csv')

In [23]:
df_cnn.head()

,Text
https://www.coursera.org/learn/python unknown 4.8 29620,cours aim teach everyon basic program comput u...
https://www.coursera.org/learn/algorithms-part1 Intermediate 4.9 2648,cours cover essenti inform everi seriou progra...
https://www.coursera.org/learn/python-data-analysis Intermediate 4.5 3836,cours introduc learner basic python program en...
https://www.coursera.org/learn/algorithmic-toolbox unknown 4.6 2355,cours cover basic algorithm techniqu idea comp...
https://www.coursera.org/learn/python-data unknown 4.8 16116,cours introduc core data structur python progr...


In [24]:
def get_closest_neighs(name, target='course'):
    if target == 'course':
        nbrs_local = nbrs_courses
    elif target == 'question':
        nbrs_local = nbrs_questions
    elif target == 'multi':
        nbrs_local = nbrs
    elif target == "topic":
        nbrs_local = nbrs_topics
    row = df_cnn.index.get_loc(name)
    distances, indices = nbrs_local.kneighbors(tfidf_matrix.getrow(row))
    indexes_similar = pd.Series(indices.flatten()).map(df_cnn.reset_index()['index'])
    result = pd.DataFrame({'distance': distances.flatten(), 'index': indexes_similar})
    
    return result

In [25]:
pred = get_closest_neighs("https://www.coursera.org/learn/python", "topic")

NameError: name 'nbrs_topics' is not defined

In [ ]:
print(pred.loc[0, "index"])

In [ ]:
pred

# kNN для топиков

In [ ]:
df_topics = pd.read_csv("df_for_knn_2.csv", index_col = 'index')
df_courses_topics = pd.read_csv("coursera_topics.csv", index_col = "topic")

In [ ]:
df_courses_topics.drop("index", inplace = True, axis = 1)

In [ ]:
df_topics.tail(10)

In [ ]:
tfidf_lda = tfidf_vectorizer.fit_transform(df_topics["Text"])
tfidf_topics = tfidf_vectorizer.transform(df_courses_topics["Text"])

In [ ]:
tfidf_topics.shape

In [ ]:
nbrs = NearestNeighbors(n_neighbors=10).fit(tfidf_lda)
nbrs_topics = NearestNeighbors(n_neighbors=10).fit(tfidf_topics)

In [ ]:
def get_closest_topic(name):
    row = df_topics.index.get_loc(name)
    distances, indices = nbrs_topics.kneighbors(tfidf_lda.getrow(row))
    indexes_similar = pd.Series(indices.flatten()).map(df_topics.reset_index()['index'])
    result = pd.DataFrame({'distance': distances.flatten(), 'index': indexes_similar})
    
    return result

In [ ]:
get_closest_topic(26497549)